In [1]:
import random
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split
import tensorflow as tf
import cv2
import os
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD

In [2]:
# load train and test dataset
def concatImages(pt1,pt2):
  e1 = cv2.imread(pt1,cv2.IMREAD_GRAYSCALE)
  e2 = cv2.imread(pt2,cv2.IMREAD_GRAYSCALE)
  return np.concatenate((e1,e2),axis=1)

# scale pixels
def prep_pixels(tensor_arr):
  # convert from integers to floats
  train_norm = tensor_arr.astype('float32')
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  # return normalized images
  return train_norm

def loadIntraClass(db_root):
  trainX = []
  trainY = []
  dirs = os.listdir(db_root)
  for dir in dirs:
    files = os.listdir(db_root+dir)
    for k1 in range(len(files)-1):
      for k2 in range(k1+1, len(files)):
        trainX.append(concatImages(f'{db_root}{dir}/{files[k1]}', f'{db_root}{dir}/{files[k2]}'))
        trainY.append([1,0])
  return trainX, trainY

def loadInterClass(db_root):
  random.seed(100)
  trainX = []
  trainY = []
  dirs = os.listdir(db_root)
  for dir1 in range(len(dirs)-1):
    files1 = os.listdir(db_root+dirs[dir1])
    for dir2 in range(dir1, len(dirs)):
      files2 = os.listdir(db_root+dirs[dir2])
      n1 = len(files1)
      n2 = len(files2)
      if(n1*n2>77):
        #generate 77 non matches
        samples = []
        while(len(samples)<77):
          k1 = random.randint(0,n1-1)
          k2 = random.randint(0,n2-1)
          if((k1,k2) not in samples):
            samples.append((k1,k2))
        for (k1,k2) in samples:
          trainX.append(concatImages(f'{db_root}{dirs[dir1]}/{files1[k1]}', f'{db_root}{dirs[dir2]}/{files2[k2]}'))
          trainY.append([0,1])
      else:
        for k1 in files1:
          for k2 in files2:
            trainX.append(concatImages(f'{db_root}{dirs[dir1]}/{k1}', f'{db_root}{dirs[dir2]}/{k2}'))
            trainY.append([0,1])
  return trainX, trainY

def load_tensors(db_root):
  X,Y = loadIntraClass(db_root)
  X1,Y1 = loadInterClass(db_root)
  X.extend(X1)
  Y.extend(Y1)
  X = tf.convert_to_tensor(X)
  X = prep_pixels(X)
  Y = tf.convert_to_tensor(Y)
  X.reshape((X.shape[0], 160, 280, 1))
  print(X.shape)
  return X,Y

def load_dataset(db_root):
  # load dataset
  trainX, trainY = load_tensors(f'{db_root}/train/edges/')
  # trainX = []
  # trainY = []
  testX, testY = load_tensors(f'{db_root}/test/edges/')
  return trainX, trainY, testX, testY


In [3]:
trainX, trainY, testX, testY = load_dataset('features')

(12856, 160, 280)
(5610, 160, 280)


In [5]:
print(trainY.shape, testY.shape)

(12856, 2) (5610, 2)


In [16]:
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(160, 280, 1)))
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dense(200, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model




# run the test harness for evaluating a model
def run_test_harness(trainX, trainY, testX, testY):
  # load dataset
  
  # prepare pixel data
  #trainX = prep_pixels(trainX)
  
  # evaluate model
  model = define_model()
  trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.1, random_state=0)
  model.fit(trainX, trainY, epochs=5, batch_size=256, validation_data=(testX,testY))
  
  print(trainX.shape, testX.shape)
  # learning curves
  #summarize_diagnostics(histories)
  # summarize estimated performance
  #model.save_weights('./tf_models/matcher2.pth')
  return model

In [5]:
model = run_test_harness(trainX, trainY, testX, testY)

Epoch 1/5
46/46 [==============================] - 57s 1s/step - loss: 0.4334 - accuracy: 0.8257 - val_loss: 0.7794 - val_accuracy: 0.7535
Epoch 2/5
46/46 [==============================] - 60s 1s/step - loss: 0.2363 - accuracy: 0.9150 - val_loss: 0.2867 - val_accuracy: 0.8919
Epoch 3/5
46/46 [==============================] - 61s 1s/step - loss: 0.1862 - accuracy: 0.9332 - val_loss: 0.2374 - val_accuracy: 0.9090
Epoch 4/5
46/46 [==============================] - 64s 1s/step - loss: 0.1561 - accuracy: 0.9443 - val_loss: 0.2197 - val_accuracy: 0.9199
Epoch 5/5
46/46 [==============================] - 68s 1s/step - loss: 0.1362 - accuracy: 0.9499 - val_loss: 0.2239 - val_accuracy: 0.9160
(11570, 160, 280) (1286, 160, 280)


In [19]:
model = define_model()
model.load_weights('./tf_models/matcher3.pth')

In [20]:
model.evaluate(testX[:2800], testY[:2800])

88/88 [==============================] - 8s 85ms/step - loss: 0.4452 - accuracy: 0.8236


[0.4451504945755005, 0.8235714435577393]

In [7]:
model.save_weights('./tf_models/matcher4.pth')

In [14]:
X = model.predict(testX[-10:])
print(X)

1/1 [==============================] - 0s 58ms/step
[[0.8165002  0.18349978]
 [0.08199222 0.91800773]
 [0.15132786 0.84867215]
 [0.45578888 0.5442111 ]
 [0.09796178 0.9020382 ]
 [0.07260024 0.92739975]
 [0.7187     0.2813    ]
 [0.6118424  0.38815764]
 [0.8584885  0.14151153]
 [0.6211367  0.3788633 ]]
